# Анализ датасета

In [63]:
import pandas as pd
from collections import defaultdict

In [64]:
df = pd.read_csv(".\BreadBasket_DMS.csv", encoding='utf-8')
days = len(df['Date'].unique())

In [65]:
df.head(10)

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam
5,2016-10-30,10:07:57,3,Cookies
6,2016-10-30,10:08:41,4,Muffin
7,2016-10-30,10:13:03,5,Coffee
8,2016-10-30,10:13:03,5,Pastry
9,2016-10-30,10:13:03,5,Bread


In [66]:
df['Item'] = df['Item'].apply(lambda x: x.lower().strip())
df['Item'] = df.loc[df['Item'] != 'none', ['Item']]
df = df.dropna()

In [67]:
len(df['Item'].unique())

94

In [68]:
df['Transaction'].max() == len(df['Transaction'].unique())

np.False_

In [69]:
unique_items = df['Item'].unique()
_dict = defaultdict(list)
for transaction_number in df['Transaction'].unique():
    products = df.loc[df['Transaction'] == transaction_number]['Item'].to_list()
    for pr in unique_items:
        if pr in products:
            _dict[pr].append(1)
        else:
            _dict[pr].append(0)
norm_df = pd.DataFrame(_dict)
norm_df

,bread,scandinavian,hot chocolate,jam,cookies,muffin,coffee,pastry,medialuna,tea,...,coffee granules,drinking chocolate spoons,christmas common,argentina night,half slice monster,gift voucher,cherry me dried fruit,mortimer,raw bars,tacos/fajita
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9460,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9461,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
9462,0,0,0,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
9463,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
norm_df.sum()

bread                    3097
scandinavian              275
hot chocolate             552
jam                       142
cookies                   515
                         ... 
gift voucher                1
cherry me dried fruit       3
mortimer                    5
raw bars                    1
tacos/fajita               11
Length: 94, dtype: int64

# Алгоритм Apriori

In [71]:
from itertools import combinations, permutations
from tqdm import tqdm

In [72]:
def count_rows(items: list[str]):
    condition = (norm_df[items[0]] == 1)
    for i in range(1, len(items)):
        condition &= (norm_df[items[i]] == 1)
    cnt = len(norm_df[condition])
    return cnt

In [73]:
k = 1
f = 4
results = []
while True:
    f_k = []
    if k == 1:
        for item in unique_items:
            if norm_df.sum()[item] >= f:
                f_k.append(item)
    else:
        if k == 2:
            combs = [x for x in combinations(results[0], r=2)]
        else:
            previous = results[k - 2]
            possible = []
            for i, elem in enumerate(previous):
                elem = sorted(elem)
                _set = set(elem[:-1])
                if any(_set.issubset(set(x)) for x in possible):
                    continue
                for j, elem1 in enumerate(previous):
                    if i == j:
                        continue
                    if _set.issubset(set(elem1)):
                        possible.append(tuple(set(elem).union(set(elem1))))
            
            combs = []
            for elem in possible:
                flag = True
                for subset in combinations(elem, r=k-1):
                    cnt = count_rows(subset)
                    if cnt < f:
                        flag = False
                        break
                if flag:
                    combs.append(elem)
        
        for elem in tqdm(combs):
            cnt = count_rows(elem)
            if cnt >= f:
                f_k.append(elem)
    
    if len(f_k) == 0:
        break
    results.append(f_k)
    k += 1
print(results[1:])

100%|██████████| 21/21 [00:00<00:00, 2100.45it/s]
0it [00:00, ?it/s]

[[('bread', 'scandinavian'), ('bread', 'hot chocolate'), ('bread', 'jam'), ('bread', 'cookies'), ('bread', 'muffin'), ('bread', 'coffee'), ('bread', 'pastry'), ('bread', 'medialuna'), ('bread', 'tea'), ('bread', 'tartine'), ('bread', 'mineral water'), ('bread', 'farm house'), ('bread', 'fudge'), ('bread', 'juice'), ('bread', "ella's kitchen pouches"), ('bread', 'frittata'), ('bread', 'hearty & seasonal'), ('bread', 'soup'), ('bread', 'smoothies'), ('bread', 'cake'), ('bread', 'coke'), ('bread', 'my-5 fruit shoot'), ('bread', 'focaccia'), ('bread', 'sandwich'), ('bread', 'alfajores'), ('bread', 'eggs'), ('bread', 'brownie'), ('bread', 'granola'), ('bread', 'keeping it local'), ('bread', 'art tray'), ('bread', 'truffles'), ('bread', 'jammie dodgers'), ('bread', 'tiffin'), ('bread', 'the nomad'), ('bread', 'bakewell'), ('bread', 'toast'), ('bread', 'scone'), ('bread', 'vegan mincepie'), ('bread', 'crisps'), ('bread', 'salad'), ('bread', 'chicken stew'), ('bread', 'spanish brunch'), ('brea

In [74]:
results.pop(0)
while type(results[0]) != tuple:
    for elem in results[0]:
        results.append(elem)
    results.pop(0)
results

[('bread', 'scandinavian'),
 ('bread', 'hot chocolate'),
 ('bread', 'jam'),
 ('bread', 'cookies'),
 ('bread', 'muffin'),
 ('bread', 'coffee'),
 ('bread', 'pastry'),
 ('bread', 'medialuna'),
 ('bread', 'tea'),
 ('bread', 'tartine'),
 ('bread', 'mineral water'),
 ('bread', 'farm house'),
 ('bread', 'fudge'),
 ('bread', 'juice'),
 ('bread', "ella's kitchen pouches"),
 ('bread', 'frittata'),
 ('bread', 'hearty & seasonal'),
 ('bread', 'soup'),
 ('bread', 'smoothies'),
 ('bread', 'cake'),
 ('bread', 'coke'),
 ('bread', 'my-5 fruit shoot'),
 ('bread', 'focaccia'),
 ('bread', 'sandwich'),
 ('bread', 'alfajores'),
 ('bread', 'eggs'),
 ('bread', 'brownie'),
 ('bread', 'granola'),
 ('bread', 'keeping it local'),
 ('bread', 'art tray'),
 ('bread', 'truffles'),
 ('bread', 'jammie dodgers'),
 ('bread', 'tiffin'),
 ('bread', 'the nomad'),
 ('bread', 'bakewell'),
 ('bread', 'toast'),
 ('bread', 'scone'),
 ('bread', 'vegan mincepie'),
 ('bread', 'crisps'),
 ('bread', 'salad'),
 ('bread', 'chicken stew

In [75]:
rules = pd.DataFrame(columns=['Условие', 'Следствие', 'Поддержка', 'Достоверность'])
# rules = {'Условие': [], 'Следствие': [], 'Поддержка': [], 'Достоверность': []}
for elem in results:
    for p in permutations(elem, r=len(elem)-1):
        condition = list(p)
        consequence = list(set(elem).difference(set(p)))
        support = count_rows(condition + consequence) / len(norm_df) * 100
        credibility = count_rows(condition + consequence) / count_rows(condition) * 100
        rules.loc[len(rules)] = [condition, consequence, support, credibility]
rules

,Условие,Следствие,Поддержка,Достоверность
0,[bread],[scandinavian],0.655045,2.001937
1,[scandinavian],[bread],0.655045,22.545455
2,[bread],[hot chocolate],1.341786,4.100743
3,[hot chocolate],[bread],1.341786,23.007246
4,[bread],[jam],0.507132,1.549887
...,...,...,...,...
933,"[scandinavian, bread]",[medialuna],0.042261,6.451613
934,"[medialuna, scandinavian]",[bread],0.042261,33.333333
935,"[medialuna, bread]",[scandinavian],0.042261,2.500000
936,"[bread, scandinavian]",[medialuna],0.042261,6.451613


In [76]:
rules['Поддержка'].max()

np.float64(9.001584786053883)

In [77]:
rules['Достоверность'].max()

np.float64(100.0)

In [78]:
rules[(rules['Поддержка'] >= 1) & (rules['Достоверность'] >= 35)]

,Условие,Следствие,Поддержка,Достоверность
132,[hot chocolate],[coffee],2.958267,50.724638
210,[cookies],[coffee],2.820919,51.844660
264,[muffin],[coffee],1.880613,48.901099
309,[pastry],[coffee],4.754358,55.214724
311,[medialuna],[coffee],3.518225,56.923077
325,[juice],[coffee],2.060222,53.424658
333,[soup],[coffee],1.584786,46.012270
339,[cake],[coffee],5.472795,52.695829
349,[sandwich],[coffee],3.824617,53.235294
351,[alfajores],[coffee],1.965135,54.069767
